In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry_convert as pc
from datetime import datetime

In [4]:
# Load the dataset
# put your file path here
file_path = 'netflix_titles.csv'
df = pd.read_csv(file_path)

In [5]:
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [7]:
# check release year
df['country'].value_counts()

country
United States                             2818
India                                      972
United Kingdom                             419
Japan                                      245
South Korea                                199
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: count, Length: 748, dtype: int64

In [8]:
df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [11]:
df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [15]:
# Checking if there is any discrepancy in "release_year" like years concatenated in a single cell
dict(df['release_year'].value_counts(), sorted=True)

{2018: np.int64(1147),
 2017: np.int64(1032),
 2019: np.int64(1030),
 2020: np.int64(953),
 2016: np.int64(902),
 2021: np.int64(592),
 2015: np.int64(560),
 2014: np.int64(352),
 2013: np.int64(288),
 2012: np.int64(237),
 2010: np.int64(194),
 2011: np.int64(185),
 2009: np.int64(152),
 2008: np.int64(136),
 2006: np.int64(96),
 2007: np.int64(88),
 2005: np.int64(80),
 2004: np.int64(64),
 2003: np.int64(61),
 2002: np.int64(51),
 2001: np.int64(45),
 1999: np.int64(39),
 1997: np.int64(38),
 2000: np.int64(37),
 1998: np.int64(36),
 1993: np.int64(28),
 1995: np.int64(25),
 1996: np.int64(24),
 1992: np.int64(23),
 1994: np.int64(22),
 1990: np.int64(22),
 1988: np.int64(18),
 1982: np.int64(17),
 1991: np.int64(17),
 1989: np.int64(16),
 1981: np.int64(13),
 1986: np.int64(13),
 1984: np.int64(12),
 1980: np.int64(11),
 1979: np.int64(11),
 1983: np.int64(11),
 1985: np.int64(10),
 1973: np.int64(10),
 1976: np.int64(9),
 1987: np.int64(8),
 1977: np.int64(7),
 1974: np.int64(7),


In [199]:
len(df), len(df_cleaned)

(8807, 7465)

In [203]:
# df_cleaned_2 = df[(df['release_year'] >= 2001) & (df['release_year'] <= 2021)].copy()
# df_cleaned_2.__len__()

In [201]:
7465/8807 *100

84.76212104008175

In [202]:
8245/8807 * 100

93.61871238787329

In [16]:
# copy of filther df from 2010 to 2021
df_cleaned = df[(df['release_year'] >= 2010) & (df['release_year'] <= 2021)].copy()

In [17]:
df_cleaned

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8798,s8799,Movie,Zed Plus,Chandra Prakash Dwivedi,"Adil Hussain, Mona Singh, K.K. Raina, Sanjay M...",India,"December 31, 2019",2014,TV-MA,131 min,"Comedies, Dramas, International Movies",A philandering small-town mechanic's political...
8800,s8801,TV Show,Zindagi Gulzar Hai,NaN,"Sanam Saeed, Fawad Khan, Ayesha Omer, Mehreen ...",Pakistan,"December 15, 2016",2012,TV-PG,1 Season,"International TV Shows, Romantic TV Shows, TV ...","Strong-willed, middle-class Kashaf and carefre..."
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."


In [18]:
# Drop columns
# df_cleaned.drop(['show_id', 'description'], axis=1, inplace=True)

In [20]:
dict(df_cleaned['country'].value_counts())

{'United States': np.int64(2300),
 'India': np.int64(731),
 'United Kingdom': np.int64(384),
 'Japan': np.int64(201),
 'South Korea': np.int64(195),
 'Canada': np.int64(171),
 'Spain': np.int64(144),
 'France': np.int64(119),
 'Mexico': np.int64(102),
 'Turkey': np.int64(94),
 'Nigeria': np.int64(93),
 'Australia': np.int64(80),
 'Brazil': np.int64(77),
 'Egypt': np.int64(76),
 'Indonesia': np.int64(70),
 'Philippines': np.int64(69),
 'Taiwan': np.int64(67),
 'Germany': np.int64(64),
 'China': np.int64(64),
 'United States, Canada': np.int64(57),
 'United Kingdom, United States': np.int64(53),
 'Thailand': np.int64(52),
 'Argentina': np.int64(50),
 'Italy': np.int64(44),
 'Canada, United States': np.int64(38),
 'Colombia': np.int64(33),
 'South Africa': np.int64(30),
 'France, Belgium': np.int64(26),
 'United States, United Kingdom': np.int64(26),
 'Singapore': np.int64(20),
 'Netherlands': np.int64(18),
 'Pakistan': np.int64(18),
 'Hong Kong': np.int64(18),
 'Malaysia': np.int64(18),


In [22]:
df_cleaned[df_cleaned['country']=='United Kingdom, United States'].head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
94,s95,Movie,Show Dogs,Raja Gosnell,"Will Arnett, Ludacris, Natasha Lyonne, Stanley...","United Kingdom, United States","September 8, 2021",2018,PG,90 min,"Children & Family Movies, Comedies",A rough and tough police dog must go undercove...
117,s118,Movie,Final Account,Luke Holland,NaN,"United Kingdom, United States","September 2, 2021",2021,PG-13,94 min,Documentaries,This documentary stitches together never-befor...
284,s285,Movie,The Kissing Booth 3,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States","August 11, 2021",2021,TV-14,114 min,"Comedies, Romantic Movies",Determined to make the most of her final summe...
348,s349,Movie,Poms,Zara Hayes,"Diane Keaton, Jacki Weaver, Pam Grier, Rhea Pe...","United Kingdom, United States","August 1, 2021",2019,PG-13,91 min,Comedies,"After moving to a retirement community, loner ..."
585,s586,Movie,Ophelia,Claire McCarthy,"Daisy Ridley, Naomi Watts, Clive Owen, George ...","United Kingdom, United States","July 1, 2021",2018,PG-13,106 min,"Dramas, Romantic Movies",In this retelling of “Hamlet” from Ophelia’s p...


In [214]:
df_cleaned['country_cleaned'] = df_cleaned['country'].apply(lambda x: x.split(',') if pd.notna(x) else x)

In [216]:
df_cleaned.reset_index(inplace=True, drop=True)
nan = []
more = []
single = []
for i in range(len(df_cleaned)):
    try:
        if len(df_cleaned['country_cleaned'][i]) > 1:
            more.append(df_cleaned['country_cleaned'][i])
        else:
            single.append(df_cleaned['country_cleaned'][i])
    except:
        nan.append(df_cleaned['country_cleaned'][i])

In [217]:
len(more), len(single), len(nan)

(1058, 5667, 740)

In [41]:
m = 0
for i in more:
    if len(i)>m:
        m = len(i)

m

12

In [218]:
df_cleaned.isnull().sum()

type                  0
country             740
date_added            0
month                 0
day                   0
year                  0
release_year          0
rating                0
duration_min          0
duration_season       0
difference            0
continent           783
country_cleaned     740
duration              0
listed_in             0
description           0
show_id               0
director           2444
cast                793
title                 0
dtype: int64

In [25]:
# keep first country in country column
# Ex. "United States, Ghana, Burkina Faso, United Kingdom" -> keep only first country == United States
# df_cleaned['country'] = 
# df_cleaned['country'].apply(lambda x: x.split(',')[0] if pd.notna(x) else x)

In [220]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

continents = []
for i in range(len(df_cleaned)):
    try:
        if len(df_cleaned['country_cleaned'][i]) == 1:
            continents.append(country_to_continent(country_name=df_cleaned['country_cleaned'][i][0]))
        else:
            continents.append("Multi-Continent")
    except:
        continents.append(np.nan)

In [221]:
df_cleaned['continent'] = continents

In [222]:
df_cleaned.head()

,type,country,date_added,month,day,year,release_year,rating,duration_min,duration_season,difference,continent,country_cleaned,duration,listed_in,description,show_id,director,cast,title
0,Movie,United States,2021-09-25,9,25,2021,2020,PG-13,90,NA,20,North America,[United States],90 min,Documentaries,"As her father nears the end of his life, filmm...",s1,Kirsten Johnson,NaN,Dick Johnson Is Dead
1,TV Show,South Africa,2021-09-24,9,24,2021,2021,TV-MA,NA,2,8,Africa,[South Africa],2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",s2,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",Blood & Water
2,TV Show,NaN,2021-09-24,9,24,2021,2021,TV-MA,NA,1,8,NaN,NaN,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,s3,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Ganglands
3,TV Show,NaN,2021-09-24,9,24,2021,2021,TV-MA,NA,1,8,NaN,NaN,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",s4,NaN,NaN,Jailbirds New Orleans
4,TV Show,India,2021-09-24,9,24,2021,2021,TV-MA,NA,2,8,Asia,[India],2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,s5,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",Kota Factory


In [57]:
# # keep first country in country column
# # Ex. "United States, Ghana, Burkina Faso, United Kingdom" -> keep only first country == United States
# df_cleaned['country'] = df_cleaned['country'].apply(lambda x: x.split(',')[0] if pd.notna(x) else x)

# # map country to continent
# # https://stackoverflow.com/questions/55910004/get-continent-name-from-country-using-pycountry
# def country_to_continent(country_name):
#     # we have missing data points so we need to use try and return None
#     try:
#         country = pc.country_name_to_country_alpha2(country_name.strip())
#         country_continent_code = pc.country_alpha2_to_continent_code(country)
#         country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
#         return country_continent_name
#     except:
#         return None

# # create 'continent'
# df_cleaned['continent'] = df_cleaned['country'].apply(country_to_continent)


In [58]:
df_cleaned.isnull().sum()

show_id               0
type                  0
title                 0
director           2451
cast                793
country             784
date_added            7
release_year          0
rating                4
duration              3
listed_in             0
description           0
country_cleaned     784
continent           784
dtype: int64

In [62]:
df_cleaned[df_cleaned['date_added'].isnull()==True]['title'].values

array(["A Young Doctor's Notebook and Other Stories",
       'Anthony Bourdain: Parts Unknown', 'Kikoriki',
       'La Familia P. Luche', 'Maron', 'Red vs. Blue',
       'The Adventures of Figaro Pho'], dtype=object)

* The above 7 titles were not available on Netflix, I think we have to drop them since the date added can't be filled from information online.

In [65]:
df_cleaned.dropna(subset=['date_added'], inplace=True)

In [66]:
df_cleaned.isnull().sum()

show_id               0
type                  0
title                 0
director           2444
cast                793
country             783
date_added            0
release_year          0
rating                4
duration              3
listed_in             0
description           0
country_cleaned     783
continent           783
dtype: int64

In [67]:
df_cleaned[df_cleaned['rating'].isnull()==True]['title'].values

array(['13TH: A Conversation with Oprah Winfrey & Ava DuVernay',
       'Gargantia on the Verdurous Planet', 'Little Lunch',
       'My Honor Was Loyalty'], dtype=object)

In [68]:
rate = {"13TH: A Conversation with Oprah Winfrey & Ava DuVernay": "TV-PG",
        "Gargantia on the Verdurous Planet": "TV-14",
        "Little Lunch": "TV-G",
        "My Honor Was Loyalty": "PG-13"}

In [88]:
for idx, val in zip(list(df_cleaned[df_cleaned['rating'].isnull()==True].index), list(rate.values())):
    df_cleaned['rating'][idx] = val

In [89]:
df_cleaned[df_cleaned['duration'].isnull()==True]['title'].values

array(['Louis C.K. 2017', 'Louis C.K.: Hilarious',
       'Louis C.K.: Live at the Comedy Store'], dtype=object)

In [90]:
dur = {}
for i in df_cleaned[df_cleaned['duration'].isnull()==True]['title'].values:
    dur[i] = ""

dur

{'Louis C.K. 2017': '',
 'Louis C.K.: Hilarious': '',
 'Louis C.K.: Live at the Comedy Store': ''}

In [91]:
df_cleaned['duration']

0          90 min
1       2 Seasons
2        1 Season
3        1 Season
4       2 Seasons
          ...    
7467      131 min
7468     1 Season
7469       96 min
7470    2 Seasons
7471      111 min
Name: duration, Length: 7465, dtype: object

In [92]:
{'Louis C.K. 2017': '74 min',
 'Louis C.K.: Hilarious': '83 min',
 'Louis C.K.: Live at the Comedy Store': '65 min'}

{'Louis C.K. 2017': '74 min',
 'Louis C.K.: Hilarious': '83 min',
 'Louis C.K.: Live at the Comedy Store': '65 min'}

In [93]:
df_cleaned[df_cleaned['duration'].isnull()==True].index

Index([4960, 5196, 5213], dtype='int64')

In [94]:
df_cleaned['duration'][4960] = '74 min'
df_cleaned['duration'][5196] = '83 min'
df_cleaned['duration'][5213] = '65 min'

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/292086631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['duration'][4960] = '74 min'
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/292086631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['duration'][5196] = '83 min'
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/292086631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [95]:
df_cleaned.isnull().sum()

show_id               0
type                  0
title                 0
director           2444
cast                793
country             783
date_added            0
release_year          0
rating                0
duration              0
listed_in             0
description           0
country_cleaned     783
continent           783
dtype: int64

In [96]:
# remove space from date_added
df_cleaned['date_added'] = df_cleaned['date_added'].str.strip()

# convert 'date_added' to datetime type
df_cleaned['date_added'] = pd.to_datetime(df_cleaned['date_added'], format='%B %d, %Y')

# extract 'month', 'day', and 'year' from 'date_added'
df_cleaned['month'] = df_cleaned['date_added'].dt.month
df_cleaned['day'] = df_cleaned['date_added'].dt.day
df_cleaned['year'] = df_cleaned['date_added'].dt.year

# calculate the difference in months
# https://stackoverflow.com/questions/4039879/best-way-to-find-the-months-between-two-dates
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# create 'difference' in months
df_cleaned['difference'] = df_cleaned.apply(lambda row: diff_month(row['date_added'], datetime(row['release_year'], 1, 1)), axis=1)

In [97]:
df_cleaned.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,country_cleaned,continent,month,day,year,difference
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",[United States],North America,9,25,2021,20
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",[South Africa],Africa,9,24,2021,8
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,9,24,2021,8
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,9,24,2021,8
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,[India],Asia,9,24,2021,8


In [99]:
df_cleaned.isnull().sum()

show_id               0
type                  0
title                 0
director           2444
cast                793
country             783
date_added            0
release_year          0
rating                0
duration              0
listed_in             0
description           0
country_cleaned     783
continent           783
month                 0
day                   0
year                  0
difference            0
dtype: int64

In [98]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7465 entries, 0 to 7471
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   show_id          7465 non-null   object        
 1   type             7465 non-null   object        
 2   title            7465 non-null   object        
 3   director         5021 non-null   object        
 4   cast             6672 non-null   object        
 5   country          6682 non-null   object        
 6   date_added       7465 non-null   datetime64[ns]
 7   release_year     7465 non-null   int64         
 8   rating           7465 non-null   object        
 9   duration         7465 non-null   object        
 10  listed_in        7465 non-null   object        
 11  description      7465 non-null   object        
 12  country_cleaned  6682 non-null   object        
 13  continent        6682 non-null   object        
 14  month            7465 non-null   int32       

In [ ]:
# next task clean where GOOD LUCK... it's a lot: 
# director         True
# cast             True
# country          True
# date_added       True
# rating           True
# duration         True

# after filling out the missing data. Put code on top or get a new data set if cleaned from excel...

In [101]:
# df_cleaned.loc[df_cleaned['director'].isnull()]
# df_cleaned.loc[df_cleaned['cast'].isnull()]
# df_cleaned.loc[df_cleaned['country'].isnull()]
# df_cleaned.loc[df_cleaned['date_added'].isnull()]
# df_cleaned.loc[df_cleaned['rating'].isnull()]
# df_cleaned.loc[df_cleaned['duration'].isnull()]

* Checking multi-continent (Maybe there are multiple countries but they might be from the same continent)

In [223]:
df_cleaned['continent'].unique()

array(['North America', 'Africa', nan, 'Asia', 'Europe',
       'Multi-Continent', 'Oceania', 'South America'], dtype=object)

In [224]:
# df_cleaned['continent'].value_counts()
continents_mix = {}
for i in list(df_cleaned[df_cleaned['continent']=='Multi-Continent'].index):
    tmp_cont = []
    for country in df_cleaned.loc[i]['country_cleaned']:
        if country.strip():
            try:
                tmp_cont.append(country_to_continent(country_name=country.strip()))
            except:
                print(i, country)
    continents_mix[i] = tmp_cont

6704  Vatican City


In [225]:
for k, v in continents_mix.items():
    if list(set(v)).__len__() == 1:
        df_cleaned['continent'][k] = list(set(v))[0]
    else:
        df_cleaned['continent'][k] = "Multi-Continent"

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/1207058120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['continent'][k] = list(set(v))[0]
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/1207058120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['continent'][k] = "Multi-Continent"
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/1207058120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [226]:
df_cleaned['continent'].value_counts()

continent
North America      2694
Asia               1781
Europe             1026
Multi-Continent     729
Africa              214
South America       189
Oceania              92
Name: count, dtype: int64

* Oceania is basically Australia, New Zealand and a few other pacific islands --> from internet (we can replace it to australia as well)

In [133]:
df_cleaned['type'].value_counts()

type
Movie      4978
TV Show    2487
Name: count, dtype: int64

In [139]:
df_cleaned['director'].nunique(), dict(df_cleaned['director'].value_counts(), sorted=True)

(3950,
 {'Raúl Campos, Jan Suter': np.int64(18),
  'Rajiv Chilaka': np.int64(17),
  'Suhas Kadav': np.int64(16),
  'Marcus Raboy': np.int64(16),
  'Jay Karas': np.int64(14),
  'Jay Chapman': np.int64(12),
  'Cathy Garcia-Molina': np.int64(11),
  'Don Michael Paul': np.int64(10),
  'Hakan Algül': np.int64(8),
  'Lance Bangs': np.int64(8),
  'Ryan Polito': np.int64(8),
  'Fernando Ayllón': np.int64(8),
  'Justin G. Dyck': np.int64(7),
  'Mae Czarina Cruz': np.int64(7),
  'Kunle Afolayan': np.int64(7),
  'Omoni Oboli': np.int64(7),
  'Prakash Satam': np.int64(7),
  'S.S. Rajamouli': np.int64(7),
  'Shannon Hartman': np.int64(7),
  'Ozan Açıktan': np.int64(7),
  'Joey So': np.int64(6),
  'Troy Miller': np.int64(6),
  'Vlad Yudin': np.int64(6),
  'Robert Vince': np.int64(6),
  'Sameh Abdulaziz': np.int64(6),
  'Hidenori Inoue': np.int64(6),
  'Wenn V. Deramas': np.int64(5),
  'Matt Askem': np.int64(5),
  'David Batty': np.int64(5),
  'Leslie Small': np.int64(5),
  'Hanung Bramantyo': np.int

In [153]:
# df_cleaned[df_cleaned['director']=="Raúl Campos, Jan Suter"]
df_cleaned[df_cleaned['director']=="Prakash Satam"]
# df_cleaned['country'][3217] = "United States"

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,country_cleaned,continent,month,day,year,difference
57,s78,Movie,Little Singham - Black Shadow,Prakash Satam,"Sumriddhi Shukla, Jigna Bharadwaj, Sonal Kaush...",NaN,2021-09-11,2021,TV-Y7,48 min,"Children & Family Movies, Comedies",Kid cop Little Singham loses all his superpowe...,NaN,NaN,9,11,2021,8
685,s862,Movie,Little Singham Future mein Satakli,Prakash Satam,"Anamaya Verma, Ganesh Divekar, Jigna Bharadhwa...",NaN,2021-05-20,2021,TV-Y7,47 min,"Children & Family Movies, Comedies",Little Singham and his crew travel to the year...,NaN,NaN,5,20,2021,4
1814,s2115,Movie,Little Singham: Kaal Ka Badla,Prakash Satam,"Anamaya Verma, Arushi Talwar, Ganesh Divekar, ...",NaN,2020-08-19,2020,TV-Y7,70 min,"Children & Family Movies, Comedies","Little Singham’s biggest enemy, the demon Kaal...",NaN,NaN,8,19,2020,7
2823,s3226,Movie,Little Singham: Mahabali,Prakash Satam,"Saumya Daan, Sonal Kaushal, Anamaya Verma, Gan...",NaN,2019-11-27,2019,TV-Y7,69 min,"Children & Family Movies, Comedies",In a journey back in time to the ancient city ...,NaN,NaN,11,27,2019,10
2979,s3391,Movie,Little Singham: Kaal Ki Tabaahi,Prakash Satam,NaN,NaN,2019-10-19,2019,TV-Y7,66 min,"Children & Family Movies, Comedies",Kaal is back – bigger and badder than ever! Ca...,NaN,NaN,10,19,2019,9
6370,s7318,Movie,Little Singham aur Kaal ka Mahajaal,Prakash Satam,NaN,NaN,2018-12-01,2018,TV-Y7-FV,68 min,"Children & Family Movies, Comedies","When Kaal, the vilest demon on earth, threaten...",NaN,NaN,12,1,2018,11
6372,s7320,Movie,Little Singham in London,Prakash Satam,NaN,NaN,2019-04-22,2019,TV-Y7,66 min,"Children & Family Movies, Comedies","Little Singham is in London to meet the queen,...",NaN,NaN,4,22,2019,3


In [154]:
for i in list(df_cleaned[df_cleaned['director']=='Prakash Satam'].index):
    df_cleaned['country'][i] = "India"

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/2218185922.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_cleaned['country'][i] = "India"
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/2218185922.py

In [148]:
for i in list(df_cleaned[df_cleaned['director']=='Suhas Kadav'].index):
    df_cleaned['country'][i] = "India"

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/2367921417.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_cleaned['country'][i] = "India"
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/2367921417.py

In [143]:
for i in list(df_cleaned[df_cleaned['director']=='S.S. Rajamouli'].index):
    df_cleaned['country'][i] = "India"

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/1378425768.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_cleaned['country'][i] = "India"
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/1378425768.py

In [145]:
for i in list(df_cleaned[df_cleaned['director']=="Rajiv Chilaka"].index):
    df_cleaned['country'][i] = "India"

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/702332957.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_cleaned['country'][i] = "India"
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/702332957.py:2

In [156]:
df_cleaned['rating'].value_counts(), df_cleaned['rating'].nunique()

(rating
 TV-MA       3028
 TV-14       1810
 TV-PG        737
 R            551
 TV-Y7        299
 TV-Y         298
 PG-13        277
 TV-G         203
 PG           171
 NR            67
 G             11
 TV-Y7-FV       6
 NC-17          3
 74 min         1
 84 min         1
 66 min         1
 UR             1
 Name: count, dtype: int64,
 17)

In [157]:
df_cleaned[df_cleaned['rating'].isin(['74 min', '84 min', '66 min', 'UR'])]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,country_cleaned,continent,month,day,year,difference
4960,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,2017-04-04,2017,74 min,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi...",[United States],North America,4,4,2017,3
5196,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,2016-09-16,2010,84 min,83 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...,[United States],North America,9,16,2016,80
5213,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,2016-08-15,2015,66 min,65 min,Movies,The comic puts his trademark hilarious/thought...,[United States],North America,8,15,2016,19
6873,s7989,Movie,Sex Doll,Sylvie Verheyde,"Hafsia Herzi, Ash Stymest, Karole Rocher, Paul...","United Kingdom, France",2017-07-10,2016,UR,103 min,"Dramas, International Movies, Romantic Movies",A high-end call girl falls for a handsome man ...,"[United Kingdom, France]",Europe,7,10,2017,18


In [158]:
df_cleaned['rating'][4960] = 'TV-MA'
df_cleaned['rating'][5196] = 'TV-MA'
df_cleaned['rating'][5213] = 'TV-MA'

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/3784099267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['rating'][4960] = 'TV-MA'
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/3784099267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['rating'][5196] = 'TV-MA'
/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/3784099267.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [159]:
df_cleaned['rating'].value_counts(), df_cleaned['rating'].nunique()

(rating
 TV-MA       3031
 TV-14       1810
 TV-PG        737
 R            551
 TV-Y7        299
 TV-Y         298
 PG-13        277
 TV-G         203
 PG           171
 NR            67
 G             11
 TV-Y7-FV       6
 NC-17          3
 UR             1
 Name: count, dtype: int64,
 14)

In [160]:
df_cleaned['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
'''
ratings = {
    'Movie Ratings': {
        'G': 'General Audience - All ages.',
        'PG': 'Parental Guidance - Some material not suitable for children.',
        'PG-13': 'Parents Cautioned - Some material may not be suitable for under 13.',
        'R': 'Restricted - Under 17 requires an adult guardian.',
        'NC-17': 'Adults Only - No one 17 and under admitted.',
        'NR': 'Not Rated - Older movies before rating system.',
        'UR': 'Unrated - Often direct-to-video or re-releases with added content.'
    },
    'TV Ratings': {
        'TV-Y': 'All Children - For very young children (2-6 years old).',
        'TV-Y7': 'Older Children - Suitable for 7 and up.',
        'TV-Y7-FV': 'Older Children - May contain intense fantasy violence.',
        'TV-G': 'General Audience - Suitable for all ages.',
        'TV-PG': 'Parental Guidance - May contain moderate language or violence.',
        'TV-14': 'Parents Cautioned - May contain strong language or violence.',
        'TV-MA': 'Mature Audience - Unsuitable for children under 17.'
    }
}

'''

In [162]:
df_cleaned['duration'].value_counts(), df_cleaned['duration'].nunique()

(duration
 1 Season      1673
 2 Seasons      403
 3 Seasons      194
 90 min         136
 91 min         126
               ... 
 181 min          1
 212 min          1
 204 min          1
 17 Seasons       1
 205 min          1
 Name: count, Length: 195, dtype: int64,
 195)

In [165]:
df_cleaned.reset_index(inplace=True, drop=True)

In [167]:
season = 0
minu = 0
for i in range(len(df_cleaned)):
    if df_cleaned['duration'][i].lower().__contains__("season"):
        season+=1
    elif df_cleaned['duration'][i].lower().__contains__("min"):
        minu+=1

season, minu

(2487, 4978)

In [168]:
2487+4978 == len(df_cleaned)

True

In [179]:
duration_min = []
duration_season = []
for i in range(len(df_cleaned)):
    if df_cleaned['duration'][i].lower().__contains__("season"):
        duration_season.append(df_cleaned['duration'][i].lower().strip("seasons").strip("season").strip())
    else:
        duration_season.append("NA")
    if df_cleaned['duration'][i].lower().__contains__("min"):
        duration_min.append(df_cleaned['duration'][i].lower().strip("min").strip())
    else:
        duration_min.append("NA")

In [180]:
len(duration_min), len(duration_season)

(7465, 7465)

In [181]:
df_cleaned['duration_min'] = duration_min
df_cleaned['duration_season'] = duration_season

In [198]:
df_cleaned['continent'].value_counts()

continent
North America      2693
Asia               1739
Europe             1026
Multi-Continent     729
Africa              214
South America       189
Oceania              92
Name: count, dtype: int64

In [183]:
dict(df_cleaned['listed_in'].value_counts())

{'Documentaries': np.int64(345),
 'Stand-Up Comedy': np.int64(306),
 'Dramas, International Movies': np.int64(306),
 'Dramas, Independent Movies, International Movies': np.int64(225),
 'Comedies, Dramas, International Movies': np.int64(218),
 "Kids' TV": np.int64(196),
 'Children & Family Movies': np.int64(189),
 'Documentaries, International Movies': np.int64(177),
 'Children & Family Movies, Comedies': np.int64(150),
 'Comedies, International Movies': np.int64(150),
 'Dramas, International Movies, Romantic Movies': np.int64(149),
 'Comedies, International Movies, Romantic Movies': np.int64(128),
 'Dramas, International Movies, Thrillers': np.int64(123),
 'International TV Shows, TV Dramas': np.int64(113),
 'Crime TV Shows, International TV Shows, TV Dramas': np.int64(109),
 'Dramas': np.int64(104),
 'Action & Adventure, Dramas, International Movies': np.int64(102),
 'Comedies, Dramas, Independent Movies': np.int64(102),
 'Reality TV': np.int64(93),
 'Action & Adventure': np.int64(91)

In [186]:
df_cleaned['listed_in'].unique()

array(['Documentaries', 'International TV Shows, TV Dramas, TV Mysteries',
       'Crime TV Shows, International TV Shows, TV Action & Adventure',
       'Docuseries, Reality TV',
       'International TV Shows, Romantic TV Shows, TV Comedies',
       'TV Dramas, TV Horror, TV Mysteries', 'Children & Family Movies',
       'British TV Shows, Reality TV', 'Comedies, Dramas',
       'Crime TV Shows, Docuseries, International TV Shows',
       'Dramas, International Movies',
       'Children & Family Movies, Comedies',
       'British TV Shows, Crime TV Shows, Docuseries',
       'TV Comedies, TV Dramas', 'Documentaries, International Movies',
       'Crime TV Shows, Spanish-Language TV Shows, TV Dramas',
       'Thrillers',
       'International TV Shows, Spanish-Language TV Shows, TV Action & Adventure',
       'International TV Shows, TV Action & Adventure, TV Dramas',
       'Docuseries, International TV Shows, Reality TV', 'Comedies',
       'Horror Movies, Sci-Fi & Fantasy',
       

In [189]:
df_cleaned['listed_in'].nunique()

451

In [195]:
# genres = [
#     "Action & Adventure",
#     "Anime Features",
#     "Anime Series",
#     "British TV Shows",
#     "Children & Family Movies",
#     "Classic & Cult TV",
#     "Comedies",
#     "Crime TV Shows",
#     "Documentaries",
#     "Dramas",
#     "Horror Movies",
#     "International Movies",
#     "International TV Shows",
#     "Korean TV Shows",
#     "LGBTQ Movies",
#     "Music & Musicals",
#     "Reality TV",
#     "Romantic Movies",
#     "Romantic TV Shows",
#     "Sci-Fi & Fantasy",
#     "Spanish-Language TV Shows",
#     "Sports Movies",
#     "Stand-Up Comedy & Talk Shows",
#     "Teen TV Shows",
#     "Thrillers"
# ]


In [207]:
df_cleaned = df_cleaned[['type', 'country', 'date_added', 'month', 'day', 'year', 'release_year', 'rating', 'duration_min', 'duration_season', 
                         'difference', 'continent', 'country_cleaned', 'duration', 'listed_in', 'description', 'show_id', 'director', 'cast',
                         'title']]

In [208]:
df_cleaned.columns

Index(['type', 'country', 'date_added', 'month', 'day', 'year', 'release_year',
       'rating', 'duration_min', 'duration_season', 'difference', 'continent',
       'country_cleaned', 'duration', 'listed_in', 'description', 'show_id',
       'director', 'cast', 'title'],
      dtype='object')

In [209]:
df_cleaned.head(2)

,type,country,date_added,month,day,year,release_year,rating,duration_min,duration_season,difference,continent,country_cleaned,duration,listed_in,description,show_id,director,cast,title
0,Movie,United States,2021-09-25,9,25,2021,2020,PG-13,90,NA,20,North America,[United States],90 min,Documentaries,"As her father nears the end of his life, filmm...",s1,Kirsten Johnson,NaN,Dick Johnson Is Dead
1,TV Show,South Africa,2021-09-24,9,24,2021,2021,TV-MA,NA,2,8,Africa,[South Africa],2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",s2,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",Blood & Water


In [236]:
def df_normalize_strings(df, col):
    df[col] = df[col].str.lower() # change everything to lower cases
    df[col] = df[col].fillna('missing') # make None -> missing
    df[col] = df[col].replace(np.nan, 'missing')
    df[col] = df[col].replace('none or unspecified', 'missing')
    df[col] = df[col].replace('none', 'missing')
    df[col] = df[col].replace('no', 'missing')
    df[col] = df[col].replace('#name?', 'missing')
    df[col] = df[col].replace('', 'missing') # empty -> missing
def normalize_all_columns(df):
    for col in df.columns:
        if df[col].dtype == 'object':  # Only apply to string columns
            df_normalize_strings(df, col)

In [237]:
normalize_all_columns(df_cleaned)

In [243]:
df_cleaned['duration_min'].replace("na", np.nan, inplace=True)
df_cleaned['duration_season'].replace("na", np.nan, inplace=True)

/var/folders/hs/n_n6ry4j37s9rkkh26n8m01r0000gn/T/ipykernel_42550/712719885.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['duration_season'].replace("na", np.nan, inplace=True)


In [249]:
df_cleaned['duration_min'] = df_cleaned['duration_min'].astype(float)
df_cleaned['duration_season'] = df_cleaned['duration_season'].astype(float)

In [250]:
df_cleaned.dtypes

type                       object
country                    object
date_added         datetime64[ns]
month                       int32
day                         int32
year                        int32
release_year                int64
rating                     object
duration_min              float64
duration_season           float64
difference                  int64
continent                  object
country_cleaned            object
duration                   object
listed_in                  object
description                object
show_id                    object
director                   object
cast                       object
title                      object
dtype: object

In [252]:
df_cleaned.head(2)

,type,country,date_added,month,day,year,release_year,rating,duration_min,duration_season,difference,continent,country_cleaned,duration,listed_in,description,show_id,director,cast,title
0,movie,united states,2021-09-25,9,25,2021,2020,pg-13,90.0,NaN,20,north america,missing,90 min,documentaries,"as her father nears the end of his life, filmm...",s1,kirsten johnson,missing,dick johnson is dead
1,tv show,south africa,2021-09-24,9,24,2021,2021,tv-ma,NaN,2.0,8,africa,missing,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",s2,missing,"ama qamata, khosi ngema, gail mabalane, thaban...",blood & water


In [253]:
df_cleaned.to_csv("Netflix_cleaned_1.csv", index=False)

In [ ]:
'''
WE CAN CHANGE THE ORDER OF THE QUESTIONS LATER ON TO FORM A MEANINGFUL FLOW

#1 (gopi)
Question: trend of how long does netflix take to add on the show of past vs current
Columns req: "difference", "type"
Comments: We can include the type as well (like how lon TV shows take to be added to Netflix in comparison to Movies)
Business perspective can be: If the time to be added to Netflix from the release date is lesser, it's a good point to attract the customers

#2 (ricky)
Question: are there any notictable patterns in type/content released over the past years. Which type has gotten more popular and released more
Columns req: "date_added"/"year", "type", "duration", "listed_in"
comments: we can plot something like, year/date will be on one axis and the other axis will have type | duration | listed_in (maybe a 1x3 subplots)
          we can also different plots for tv shows and movies since their duration values are different.
Business perspective: if there are more tv shows than movies, it indicates that netflix is trying increase the watch hours of the users since,
                         tv shows have more watch hours compared to movies (need to figure out, can't think of any good idea here).

#3 (victor)
Question: what age group are they targeting distrution of rating
Columns req: "rating", count of "type", "year" (maybe to show the trend in target audience)
comments: 
Business perspective: more content in the rating of "anyone can watch sort of type" means that the range and count of target audience
                     is more thus increasing the subscriptions (??? not quite sure here as well)

#4 (gopi)
Question: Global visualization
Columns req: "continent", ("country"),
business perspective: netflix catering to global audience

#5
Question: key directors and cast in the netflix ecosystem.
comments: Network graph of top directors/csat and their genres. (optional-not sure if this would work out, keeping it for now)

#6 (victor)
Question: use quarterly data to get counts of type released and see what types of shows are popular during which season
comments: Since most people have holidays in Dec/Jan, is the content more likely around tht season?
'''

In [227]:
df_cleaned.isnull().sum()

type                  0
country             740
date_added            0
month                 0
day                   0
year                  0
release_year          0
rating                0
duration_min          0
duration_season       0
difference            0
continent           740
country_cleaned     740
duration              0
listed_in             0
description           0
show_id               0
director           2444
cast                793
title                 0
dtype: int64